In [14]:
!pip install pyspark==3.5.0 google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()
print("✅ GCP Auth complete!")


✅ GCP Auth complete!


In [15]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project="spheric-wonder-469917-i9")  # FIXED project
query = """
SELECT date, dist_id, sales, dpu
FROM `spheric-wonder-469917-i9.paul_gcp_etl_demo.gcp_sales`
LIMIT 10000
"""
df_pandas = client.query(query).to_dataframe()
print(f"✅ Loaded {len(df_pandas)} rows!")
df_pandas.head()


✅ Loaded 5 rows!


,date,dist_id,sales,dpu
0,2026-02-11,distB,550000,1.920
1,2026-02-10,distA,700000,2.050
2,2026-02-11,distA,720000,2.100
3,2026-02-12,distA,744000,2.075
4,2026-02-12,distB,588000,1.895


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

spark = SparkSession.builder.appName("SalesRanking").getOrCreate()
df = spark.createDataFrame(df_pandas)

window_spec = Window.partitionBy("dist_id").orderBy(col("sales").desc())
df_ranked = df.withColumn("sales_rank", row_number().over(window_spec))

print("🎯 RANKED SALES:")
df_ranked.show(20)


🎯 RANKED SALES:
+----------+-------+------+-----+----------+
|      date|dist_id| sales|  dpu|sales_rank|
+----------+-------+------+-----+----------+
|2026-02-12|  distA|744000|2.075|         1|
|2026-02-11|  distA|720000|  2.1|         2|
|2026-02-10|  distA|700000| 2.05|         3|
|2026-02-12|  distB|588000|1.895|         1|
|2026-02-11|  distB|550000| 1.92|         2|
+----------+-------+------+-----+----------+



In [17]:
# Scale to FULL dataset
df_top3 = df_ranked.filter(col("sales_rank") <= 3)
print("🏆 TOP 3 DISTRIBUTORS BY SALES:")
df_top3.orderBy("dist_id", "sales_rank").show(50)

# Save to BigQuery
df_top3.write \
  .format("bigquery") \
  .option("table", "spheric-wonder-469917-i9.paul_gcp_etl_demo.sales_ranked") \
  .mode("overwrite") \
  .save()
print("💾 Ranked data saved to BigQuery!")


🏆 TOP 3 DISTRIBUTORS BY SALES:
+----------+-------+------+-----+----------+
|      date|dist_id| sales|  dpu|sales_rank|
+----------+-------+------+-----+----------+
|2026-02-12|  distA|744000|2.075|         1|
|2026-02-11|  distA|720000|  2.1|         2|
|2026-02-10|  distA|700000| 2.05|         3|
|2026-02-12|  distB|588000|1.895|         1|
|2026-02-11|  distB|550000| 1.92|         2|
+----------+-------+------+-----+----------+



Py4JJavaError: An error occurred while calling o109.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: bigquery. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:248)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.i

In [18]:
# Convert PySpark → Pandas → BigQuery (GUARANTEED)
df_pandas_ranked = df_ranked.toPandas()
print(f"✅ {len(df_pandas_ranked)} ranked rows ready!")

# Save to BigQuery via pandas
from google.cloud import bigquery
client = bigquery.Client(project="spheric-wonder-469917-i9")

table_id = "spheric-wonder-469917-i9.paul_gcp_etl_demo.sales_ranked"
job = client.load_table_from_dataframe(df_pandas_ranked, table_id)
job.result()  # Wait for completion
print("💾 Ranked data SAVED to BigQuery!")


✅ 5 ranked rows ready!
💾 Ranked data SAVED to BigQuery!
